<a href="https://colab.research.google.com/github/roscoekerby/google-colab/blob/main/Dynamic_Podcasts_Episodes_Scraper_2025_02_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Imports

In [1]:
!pip install selenium
!pip install chromedriver-autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 26.9 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt-get install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y
!wget https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!mv chromedriver /usr/local/bin/
!chmod +x /usr/local/bin/chromedriver


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,692 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,140 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,832 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,542 kB]
Get

In [3]:
!pip install webdriver-manager

# **Scrape Apple 2024-10-28**

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
import time
import json

def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.binary_location = '/usr/bin/google-chrome'

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1920, 1080)  # Set a larger window size
    return driver

def wait_for_episodes(driver, current_count):
    try:
        WebDriverWait(driver, 10).until(
            lambda x: len(x.find_elements(By.CSS_SELECTOR, "li.svelte-8rlk6b")) > current_count
        )
        return True
    except TimeoutException:
        return False

def scroll_to_bottom(driver):
    previous_episode_count = 0
    no_new_content_count = 0
    max_attempts = 10  # Increased max attempts

    while no_new_content_count < max_attempts:
        # Get current episode count
        current_episodes = driver.find_elements(By.CSS_SELECTOR, "li.svelte-8rlk6b")
        current_count = len(current_episodes)

        if current_count == previous_episode_count:
            no_new_content_count += 1
        else:
            no_new_content_count = 0

        # Scroll in smaller increments
        if current_episodes:
            for i in range(max(0, len(current_episodes)-5), len(current_episodes)):
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", current_episodes[i])
                    time.sleep(0.5)
                except:
                    continue

        time.sleep(2)

        new_content_loaded = wait_for_episodes(driver, current_count)
        if not new_content_loaded:
            no_new_content_count += 1

        previous_episode_count = current_count
        print(f"Current episode count: {current_count}")

def extract_episode_info(episode, index):
    """Extract information from an episode element with enhanced retry logic."""
    max_retries = 5  # Increased retries
    for attempt in range(max_retries):
        try:
            # Updated selectors based on the actual HTML structure
            title_element = WebDriverWait(episode, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "span.episode-details__title-text"))
            )
            title = title_element.text

            link_element = episode.find_element(By.CSS_SELECTOR, "a.link-action")
            link = link_element.get_attribute('href')

            # Updated selector for description - looking for the div instead of p tag
            description_element = episode.find_element(By.CSS_SELECTOR, "div.episode-details__summary .multiline-clamp__text")
            description = description_element.text

            # Get duration if available
            try:
                duration_element = episode.find_element(By.CSS_SELECTOR, "div.progress-time")
                duration = duration_element.text
            except NoSuchElementException:
                duration = "Unknown"

            # Get publication date if available
            try:
                date_element = episode.find_element(By.CSS_SELECTOR, "p.episode-details__published-date")
                publish_date = date_element.text
            except NoSuchElementException:
                publish_date = "Unknown"

            if not all([title, link]):  # We'll accept missing description as some episodes might not have one
                raise NoSuchElementException("Missing title or link data")

            return {
                'index': index,
                'title': title,
                'link': link,
                'description': description,
                'duration': duration,
                'publish_date': publish_date
            }
        except (StaleElementReferenceException, NoSuchElementException) as e:
            if attempt == max_retries - 1:
                print(f"Failed to extract episode at index {index} after {max_retries} attempts: {str(e)}")
            time.sleep(1)
            continue
    return None

def scrape_episodes(url):
    episode_data = []
    driver = init_driver()

    try:
        driver.get(url)

        # Wait for initial load
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "li.svelte-8rlk6b"))
        )

        # Take a screenshot to debug
        driver.save_screenshot("initial_load.png")

        # Scroll to load all episodes
        scroll_to_bottom(driver)

        # Get all episodes after scrolling
        episode_elements = driver.find_elements(By.CSS_SELECTOR, "li.svelte-8rlk6b")
        print(f"Total episodes found: {len(episode_elements)}")

        # Extract information from each episode
        for index, episode in enumerate(episode_elements):
            episode_info = extract_episode_info(episode, index)
            if episode_info:
                episode_data.append(episode_info)
                print(f"Scraped ({index + 1}/{len(episode_elements)}): {episode_info['title']}")
            else:
                print(f"Failed to scrape episode at index {index}")

        # Sort episodes by index to maintain order
        episode_data.sort(key=lambda x: x['index'])

        # Check for missing episodes
        titles = [ep['title'] for ep in episode_data]
        for i in range(1, 40):  # Adjust range based on expected episode count
            episode_num = f"TMGP Ep {str(i).zfill(2)}"
            if not any(episode_num in title for title in titles):
                print(f"WARNING: Potentially missing {episode_num}")

        # Save data to JSON file
        with open("podcast_episodes.json", "w", encoding="utf-8") as f:
            json.dump(episode_data, f, ensure_ascii=False, indent=4)

        print(f"Successfully saved data to podcast_episodes.json")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        driver.quit()

    return episode_data

if __name__ == "__main__":
    url = "https://podcasts.apple.com/za/podcast/the-muscle-growth-podcast/id1717906577/episodes"
    apple_episodes = scrape_episodes(url)
    print(f"\nTotal episodes successfully scraped: {len(apple_episodes)}")

    # Print all episode titles in order
    print("\nAll scraped episodes:")
    for ep in apple_episodes:
        print('*' * 80)
        print(f"Title: {ep['title']}")
        print(f"Link: {ep['link']}")
        print(f"Duration: {ep['duration']}")
        print(f"Published: {ep['publish_date']}")
        print(f"Description: {ep['description'][:150]}...")  # Print first 150 chars of description
        print('*' * 80)

Current episode count: 25
Current episode count: 38
Current episode count: 38
Current episode count: 38
Current episode count: 38
Current episode count: 38
Current episode count: 38
Total episodes found: 38
Scraped (1/38): TMGP Ep 38 with NO-BS strength and fitness coach Cillian O’Connor AKA DYSFUNCTIONALPATTERNS
Scraped (2/38): TMGP Ep 37 with men’s physique winner, bodybuilder, and lifestyle coach - Jonathan Kantor
Scraped (3/38): TMGP Ep 36 with Emanuel Pescari: Austria’s Strongest Man, MULTIPLE WORLD RECORD TITLE HOLDER & INTERNATIONAL STRONGMAN SUPERSTAR Competitor part 2
Scraped (4/38): TMGP Ep 35 with Emanuel Pescari: Austria’s Strongest Man and Elite Strongman Competitor part 1
Scraped (5/38): TMGP Ep 34 with health coach, wellness consultant mindset expert Jeffrey Siegel part 2
Scraped (6/38): TMGP Ep 33 with health coach, wellness consultant, and mindset expert Jeffrey Siegel
Scraped (7/38): TMGP Ep 32 with health coach and diabetes expert Ghamdan Al-Areeky
Scraped (8/38): TM

# **Scrape Spotify**

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
import time
import json
import os
import re
from datetime import datetime

def init_driver():
    """Initialize and return a web driver instance."""
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--incognito')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

    # Disable webdriver mode to avoid detection
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Execute CDP commands to prevent detection
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'})
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
        Object.defineProperty(navigator, 'webdriver', {
          get: () => undefined
        })
      """
    })

    return driver

def load_more_episodes(driver, max_attempts=15):
    """Click 'See more episodes' button until all episodes are loaded."""
    attempt = 0
    wait = WebDriverWait(driver, 10)

    while attempt < max_attempts:
        try:
            # Wait for page to load completely
            time.sleep(3)

            # Try to find the "See more episodes" button
            load_more_button = None
            try:
                # Try multiple possible selectors for the "load more" button
                selectors = [
                    "button[data-encore-id='buttonTertiary'][aria-expanded='false']",
                    "button[data-encore-id='buttonTertiary'][data-testid='see-more']",
                    "button[data-encore-id='buttonTertiary'][data-testid='show-more']",
                    "button.Button-sc-1dqy08n-0",
                    "button[data-encore-id='buttonTertiary']"
                ]

                for selector in selectors:
                    try:
                        load_more_button = driver.find_element(By.CSS_SELECTOR, selector)
                        # Check if the button contains text like "See more episodes"
                        if load_more_button.is_displayed() and (
                            "more" in load_more_button.text.lower() or
                            "show" in load_more_button.text.lower() or
                            "see" in load_more_button.text.lower() or
                            len(load_more_button.text.strip()) == 0  # Sometimes button has no text
                        ):
                            break
                    except:
                        continue

                if not load_more_button or not load_more_button.is_displayed():
                    print("No more episodes to load or button not found.")
                    return False

                # Scroll to the button to make sure it's visible
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button)
                time.sleep(1)

                # Click the button
                driver.execute_script("arguments[0].click();", load_more_button)
                print(f"Clicked 'Load more' button (attempt {attempt+1})")

                # Wait for new content to load
                time.sleep(3)
                attempt += 1

            except (TimeoutException, NoSuchElementException) as e:
                print(f"Could not find 'Load more' button: {e}")
                return False

        except Exception as e:
            print(f"Error while loading more episodes: {e}")
            attempt += 1
            time.sleep(2)

    return False

def extract_episodes(driver):
    """Extract all podcast episodes from the page."""
    episodes = []

    try:
        # First, make sure we're on the "All episodes" tab
        try:
            tabs = driver.find_elements(By.CSS_SELECTOR, "button[role='tab']")
            for tab in tabs:
                if "episode" in tab.text.lower() and "all" in tab.text.lower():
                    driver.execute_script("arguments[0].click();", tab)
                    print("Clicked on 'All episodes' tab")
                    time.sleep(3)
                    break
        except Exception as e:
            print(f"Could not find or click 'All episodes' tab: {e}")

        # Load all episodes
        load_more_episodes(driver)

        # Find all episode containers
        episode_elements = driver.find_elements(By.CSS_SELECTOR, "[data-testid^='episode-']")
        print(f"Found {len(episode_elements)} episode elements")

        for i, episode in enumerate(episode_elements):
            try:
                # Extract episode data
                episode_data = {}

                # Extract title
                try:
                    title_elem = episode.find_element(By.CSS_SELECTOR, "a[data-encore-id='listRowTitle']")
                    episode_data['title'] = title_elem.text.strip()
                    episode_data['url'] = title_elem.get_attribute('href')
                except Exception as e:
                    print(f"Error extracting title for episode {i+1}: {e}")
                    episode_data['title'] = "Title not found"
                    episode_data['url'] = ""

                # Extract description
                try:
                    description_elem = episode.find_element(By.CSS_SELECTOR, "p[data-encore-id='listRowDetails']")
                    episode_data['description'] = description_elem.text.strip()
                except Exception as e:
                    print(f"Error extracting description for episode {i+1}: {e}")
                    episode_data['description'] = "Description not found"

                # Extract date and duration
                try:
                    details = episode.find_elements(By.CSS_SELECTOR, "p[data-encore-id='text']")
                    if len(details) >= 1:
                        episode_data['published_date'] = details[0].text.strip()
                    if len(details) >= 2:
                        episode_data['duration'] = details[1].text.strip()
                except Exception as e:
                    print(f"Error extracting date/duration for episode {i+1}: {e}")
                    episode_data['published_date'] = ""
                    episode_data['duration'] = ""

                # Extract explicit tag if it exists
                try:
                    explicit = episode.find_element(By.CSS_SELECTOR, "span[aria-label='Explicit']")
                    episode_data['explicit'] = True
                except NoSuchElementException:
                    episode_data['explicit'] = False

                # Extract episode number if available in the title
                try:
                    ep_match = re.search(r'(?i)ep(?:isode)?\s*(\d+)', episode_data['title'])
                    if ep_match:
                        episode_data['episode_number'] = ep_match.group(1)
                    else:
                        episode_data['episode_number'] = ""
                except:
                    episode_data['episode_number'] = ""

                episodes.append(episode_data)
                print(f"Extracted episode {i+1}: {episode_data['title']}")

            except Exception as e:
                print(f"Error processing episode {i+1}: {e}")

    except Exception as e:
        print(f"Error extracting episodes: {e}")

    return episodes

def save_to_json(data, filename="spotify_podcast_episodes.json"):
    """Save the extracted data to a JSON file."""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(f"Data successfully saved to {filename}")
        return True
    except Exception as e:
        print(f"Error saving data to file: {e}")
        return False

def scrape_spotify_podcast(url):
    """Main function to scrape podcast episodes from Spotify."""
    driver = None
    try:
        print(f"Starting to scrape podcast episodes from: {url}")
        driver = init_driver()

        # Open the URL
        driver.get(url)
        print("Page loaded successfully")

        # Wait for the page to load completely
        time.sleep(5)

        # Take a screenshot for debugging
        driver.save_screenshot("spotify_page.png")
        print("Screenshot saved as spotify_page.png")

        # Extract episode data
        episodes = extract_episodes(driver)

        # Save the data to a JSON file
        current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"spotify_podcast_episodes_{current_time}.json"
        save_to_json(episodes, filename)

        # Print summary
        print(f"\nTotal episodes scraped: {len(episodes)}")
        if episodes:
            print("\nFirst episode details:")
            for key, value in episodes[0].items():
                if key == 'description':
                    print(f"{key}: {value[:100]}..." if len(value) > 100 else f"{key}: {value}")
                else:
                    print(f"{key}: {value}")

            print("\nLast episode details:")
            for key, value in episodes[-1].items():
                if key == 'description':
                    print(f"{key}: {value[:100]}..." if len(value) > 100 else f"{key}: {value}")
                else:
                    print(f"{key}: {value}")

        return episodes

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

    finally:
        if driver:
            driver.quit()
            print("Browser closed")

if __name__ == "__main__":
    # Replace with the URL of the podcast you want to scrape
    podcast_url = "https://open.spotify.com/show/0TBy1wFQocjsf4MGqPVNKV"
    episodes = scrape_spotify_podcast(podcast_url)

    if episodes:
        print(f"\nSuccessfully scraped {len(episodes)} episodes.")
    else:
        print("Failed to scrape episodes.")

Starting to scrape podcast episodes from: https://open.spotify.com/show/0TBy1wFQocjsf4MGqPVNKV
Page loaded successfully
Screenshot saved as spotify_page.png
Clicked 'Load more' button (attempt 1)
Clicked 'Load more' button (attempt 2)
Clicked 'Load more' button (attempt 3)
Clicked 'Load more' button (attempt 4)
Clicked 'Load more' button (attempt 5)
Clicked 'Load more' button (attempt 6)
Clicked 'Load more' button (attempt 7)
Clicked 'Load more' button (attempt 8)
Clicked 'Load more' button (attempt 9)
Clicked 'Load more' button (attempt 10)
Clicked 'Load more' button (attempt 11)
Clicked 'Load more' button (attempt 12)
Clicked 'Load more' button (attempt 13)
Clicked 'Load more' button (attempt 14)
Clicked 'Load more' button (attempt 15)
Found 0 episode elements
Data successfully saved to spotify_podcast_episodes_20250417_190853.json

Total episodes scraped: 0
Browser closed
Failed to scrape episodes.


In [5]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time

# Function to initialize the WebDriver
def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.binary_location = '/usr/bin/google-chrome'

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Function to click elements using CSS selectors with a retry mechanism
def click_element(driver, selector, max_retries=5, wait_time=3):
    retries = 0
    while retries < max_retries:
        try:
            element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
            )
            driver.execute_script("arguments[0].scrollIntoView();", element)  # Ensure button is visible
            element.click()
            time.sleep(wait_time)  # Add a wait time to load additional episodes
            return True
        except Exception as e:
            retries += 1
            print(f"Retry {retries}/{max_retries} - Failed to click: {e}")
            time.sleep(wait_time)
    print(f"Failed to click on element after {max_retries} retries.")
    return False

# Function to load all episodes by clicking "Load more episodes" until it's unavailable
def load_all_episodes(driver):
    load_more_selector = 'button[data-encore-id="chip"][role="checkbox"]'
    while click_element(driver, load_more_selector):
        print("Clicked 'Load more episodes' button.")
        time.sleep(3)  # Delay for loading new episodes

# Function to extract episode links from the infinite scroll list
def get_episode_links(driver):
    episode_and_show_list = []
    infinite_scroll_list = driver.find_element(By.CSS_SELECTOR, '[data-testid="infinite-scroll-list"]')
    episode_links = infinite_scroll_list.find_elements(By.TAG_NAME, 'a')

    for episode_link in episode_links:
        episode_href = episode_link.get_attribute('href')
        if episode_href and 'show' not in episode_href:
            print("Episode Link:", episode_href)
            episode_and_show_list.append(episode_href)
    return episode_and_show_list

# Main execution
if __name__ == "__main__":
    url = "https://open.spotify.com/show/0TBy1wFQocjsf4MGqPVNKV"
    driver = init_driver()
    driver.get(url)

    try:
        # Load all episodes by clicking "Load more episodes" until no more episodes can be loaded
        load_all_episodes(driver)

        # Extract and print episode links
        spotify_episodes = get_episode_links(driver)
        print("Total Episodes Found:", len(spotify_episodes))

    finally:
        driver.quit()


Retry 1/5 - Failed to click: Message: element click intercepted: Element <button class="LegacyChip__LegacyChipComponent-sc-tzfq94-0 fJQBhR encore-text-body-small" role="checkbox" aria-checked="false" data-encore-id="chip">...</button> is not clickable at point (398, 16). Other element would receive the click: <header data-testid="topbar" aria-label="Top bar and user menu" class="facDIsOQo9q7kiWc4jSg">...</header>
  (Session info: chrome=135.0.7049.95)
Stacktrace:
#0 0x567c5430fcea <unknown>
#1 0x567c53dc05f0 <unknown>
#2 0x567c53e1887c <unknown>
#3 0x567c53e166dc <unknown>
#4 0x567c53e13d72 <unknown>
#5 0x567c53e13467 <unknown>
#6 0x567c53e05b3d <unknown>
#7 0x567c53e37662 <unknown>
#8 0x567c53e054ba <unknown>
#9 0x567c53e3782e <unknown>
#10 0x567c53e5d660 <unknown>
#11 0x567c53e37433 <unknown>
#12 0x567c53e03ea3 <unknown>
#13 0x567c53e04b01 <unknown>
#14 0x567c542d4b3b <unknown>
#15 0x567c542d8a21 <unknown>
#16 0x567c542bbc32 <unknown>
#17 0x567c542d9594 <unknown>
#18 0x567c5429feef <

In [7]:
import requests
import json
import re
from bs4 import BeautifulSoup
import time

def get_podcast_episodes(show_id):
    """
    Get podcast episodes using direct API request instead of browser automation
    """
    base_url = f"https://open.spotify.com/show/{show_id}"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Cache-Control': 'max-age=0'
    }

    # First get the main page to extract any tokens needed
    print(f"Requesting podcast page: {base_url}")
    response = requests.get(base_url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to get podcast page: {response.status_code}")
        return []

    print("Successfully retrieved podcast page")

    # Parse the initial HTML to extract the Spotify app initial state data
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the script tag containing initial data
    scripts = soup.find_all('script')
    initial_data = None

    for script in scripts:
        if script.string and 'Spotify.Entity' in script.string:
            print("Found Spotify.Entity data in script tag")
            # Extract JSON data from the script content
            matches = re.search(r'Spotify\.Entity\s*=\s*(\{.+\});', script.string, re.DOTALL)
            if matches:
                try:
                    initial_data = json.loads(matches.group(1))
                    print("Successfully extracted entity data")
                    break
                except json.JSONDecodeError:
                    print("Failed to parse entity data")

    # Try another approach if the first one didn't work
    if not initial_data:
        for script in scripts:
            if script.string and '__PRELOADED_STATE__' in script.string:
                print("Found __PRELOADED_STATE__ data in script tag")
                matches = re.search(r'__PRELOADED_STATE__\s*=\s*(\{.+\});', script.string, re.DOTALL)
                if matches:
                    try:
                        initial_data = json.loads(matches.group(1))
                        print("Successfully extracted preloaded state data")
                        break
                    except json.JSONDecodeError:
                        print("Failed to parse preloaded state data")

    # Save the HTML for debugging
    with open('spotify_html.html', 'w', encoding='utf-8') as f:
        f.write(response.text)
    print("Saved HTML to spotify_html.html for debugging")

    episodes = []

    # If we couldn't get the initial data, try a different approach by parsing the HTML directly
    if not initial_data:
        print("Couldn't extract initial data, parsing HTML directly")

        # Look for episode links
        episode_links = soup.select('a[href*="/episode/"]')
        print(f"Found {len(episode_links)} episode links in HTML")

        for link in episode_links:
            episode_data = {
                'title': link.get_text().strip(),
                'url': link.get('href'),
                'description': '',
                'duration': '',
                'published_date': '',
                'explicit': False,
                'episode_number': ''
            }

            # Try to extract episode number from title
            ep_match = re.search(r'(?i)ep(?:isode)?\s*(\d+)', episode_data['title'])
            if ep_match:
                episode_data['episode_number'] = ep_match.group(1)

            episodes.append(episode_data)
            print(f"Added episode: {episode_data['title']}")

    # Save data to JSON
    if episodes:
        with open(f'spotify_podcast_{show_id}.json', 'w', encoding='utf-8') as f:
            json.dump(episodes, f, ensure_ascii=False, indent=4)
        print(f"Saved {len(episodes)} episodes to spotify_podcast_{show_id}.json")

        # Print the first and last episodes
        print("\nFirst episode:")
        for key, value in episodes[0].items():
            print(f"{key}: {value}")

        print("\nLast episode:")
        for key, value in episodes[-1].items():
            print(f"{key}: {value}")
    else:
        print("No episodes found")

    return episodes

if __name__ == "__main__":
    # The show ID is the last part of your URL
    show_id = "0TBy1wFQocjsf4MGqPVNKV"
    episodes = get_podcast_episodes(show_id)
    print(f"\nSuccessfully found {len(episodes)} episodes")

Requesting podcast page: https://open.spotify.com/show/0TBy1wFQocjsf4MGqPVNKV
Successfully retrieved podcast page
Saved HTML to spotify_html.html for debugging
Couldn't extract initial data, parsing HTML directly
Found 0 episode links in HTML
No episodes found

Successfully found 0 episodes


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
import time
import json
import os
import re
from datetime import datetime

def init_driver():
    """Initialize and return a web driver instance."""
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--incognito')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

    # Disable webdriver mode to avoid detection
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Execute CDP commands to prevent detection
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'})
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
        Object.defineProperty(navigator, 'webdriver', {
          get: () => undefined
        })
      """
    })

    return driver

def click_load_more(driver, max_attempts=15):
    """Click 'Load more episodes' button."""
    attempt = 0

    while attempt < max_attempts:
        try:
            # Look specifically for the "Load more episodes" button at the bottom
            load_more_buttons = driver.find_elements(By.XPATH, "//button[contains(., 'Load more episodes')]")

            if not load_more_buttons:
                print("No 'Load more episodes' button found, all episodes may be loaded")
                return False

            button = load_more_buttons[0]
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
            time.sleep(1)

            # Take a screenshot before clicking
            driver.save_screenshot(f"before_click_{attempt}.png")

            driver.execute_script("arguments[0].click();", button)
            print(f"Clicked 'Load more episodes' button (attempt {attempt+1})")

            # Wait for new content to load
            time.sleep(3)
            attempt += 1

        except Exception as e:
            print(f"Error clicking 'Load more episodes' button: {e}")
            attempt += 1
            time.sleep(2)

    return False

def extract_episodes(driver):
    """Extract podcast episodes using the correct selectors from the HTML structure."""
    episodes = []

    try:
        # Click the load more button until all episodes are loaded
        click_load_more(driver)

        # Save the page source for debugging
        with open("spotify_page_source.html", "w", encoding="utf-8") as f:
            f.write(driver.page_source)

        # Take a screenshot after loading episodes
        driver.save_screenshot("after_loading_episodes.png")

        # Find all episode containers
        episode_containers = driver.find_elements(By.CSS_SELECTOR, "div.T9iBYqbERZHdwDl0U2tC")
        print(f"Found {len(episode_containers)} episode containers")

        for i, container in enumerate(episode_containers):
            try:
                episode_data = {}

                # Extract title - this selector matches what's in your HTML
                try:
                    title_elem = container.find_element(By.CSS_SELECTOR, "span.episode-details__title-text, span.ListRowTitle__LineClamp-sc-1xe2if1-0")
                    episode_data['title'] = title_elem.text.strip()
                except NoSuchElementException:
                    # Try alternative selector
                    try:
                        title_elem = container.find_element(By.CSS_SELECTOR, "a[data-encore-id='listRowTitle']")
                        episode_data['title'] = title_elem.text.strip()
                    except:
                        episode_data['title'] = "Title not found"

                # Extract link
                try:
                    link_elem = container.find_element(By.CSS_SELECTOR, "a.link-action, a[data-encore-id='listRowTitle']")
                    episode_data['url'] = link_elem.get_attribute('href')
                except:
                    episode_data['url'] = ""

                # Extract description
                try:
                    desc_elem = container.find_element(By.CSS_SELECTOR, "span.ListRowDetails__LineClamp-sc-sozu4l-1")
                    episode_data['description'] = desc_elem.text.strip()
                except:
                    try:
                        # Alternative selector for description
                        desc_elem = container.find_element(By.CSS_SELECTOR, "div.episode-details__summary, p[data-encore-id='listRowDetails']")
                        episode_data['description'] = desc_elem.text.strip()
                    except:
                        episode_data['description'] = "Description not found"

                # Extract date and duration
                try:
                    date_elem = container.find_element(By.CSS_SELECTOR, "p.episode-details__published-date, p._q93agegdE655O5zPz6l")
                    episode_data['published_date'] = date_elem.text.strip()
                except:
                    episode_data['published_date'] = ""

                try:
                    duration_elem = container.find_element(By.CSS_SELECTOR, "div.progress-time, span.UyzJidwrGk3awngSGIwv")
                    episode_data['duration'] = duration_elem.text.strip()
                except:
                    episode_data['duration'] = ""

                # Extract explicit tag if it exists
                try:
                    explicit = container.find_element(By.CSS_SELECTOR, "span[aria-label='Explicit'], span.SgFtsvn3upY_tG6mnt4n")
                    episode_data['explicit'] = True
                except:
                    episode_data['explicit'] = False

                # Extract episode number
                if 'title' in episode_data and episode_data['title'] != "Title not found":
                    ep_match = re.search(r'(?i)ep(?:isode)?\s*(\d+)', episode_data['title'])
                    if ep_match:
                        episode_data['episode_number'] = ep_match.group(1)
                    else:
                        episode_data['episode_number'] = ""

                episodes.append(episode_data)
                print(f"Extracted episode {i+1}: {episode_data['title']}")

            except Exception as e:
                print(f"Error extracting data for episode {i+1}: {e}")

    except Exception as e:
        print(f"Error in extract_episodes: {e}")

    return episodes

def save_to_json(data, filename="spotify_podcast_episodes.json"):
    """Save the extracted data to a JSON file."""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(f"Data successfully saved to {filename}")
        return True
    except Exception as e:
        print(f"Error saving data to file: {e}")
        return False

def scrape_spotify_podcast(url):
    """Main function to scrape podcast episodes."""
    driver = None
    try:
        print(f"Starting to scrape podcast episodes from: {url}")
        driver = init_driver()

        # Open the URL
        driver.get(url)
        print("Page loaded successfully")

        # Wait for the page to load completely
        time.sleep(10)

        # Take a screenshot for debugging
        driver.save_screenshot("spotify_page_initial.png")

        # Extract episode data
        episodes = extract_episodes(driver)

        # Save the data to a JSON file
        current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"spotify_podcast_episodes_{current_time}.json"
        save_to_json(episodes, filename)

        # Print summary
        print(f"\nTotal episodes scraped: {len(episodes)}")
        if episodes:
            print("\nFirst episode details:")
            for key, value in episodes[0].items():
                if key == 'description':
                    print(f"{key}: {value[:100]}..." if len(value) > 100 else f"{key}: {value}")
                else:
                    print(f"{key}: {value}")

            print("\nLast episode details:")
            for key, value in episodes[-1].items():
                if key == 'description':
                    print(f"{key}: {value[:100]}..." if len(value) > 100 else f"{key}: {value}")
                else:
                    print(f"{key}: {value}")

        return episodes

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

    finally:
        if driver:
            driver.quit()
            print("Browser closed")

if __name__ == "__main__":
    # The URL of your podcast
    podcast_url = "https://open.spotify.com/show/0TBy1wFQocjsf4MGqPVNKV"
    episodes = scrape_spotify_podcast(podcast_url)

    if episodes:
        print(f"\nSuccessfully scraped {len(episodes)} episodes.")
    else:
        print("Failed to scrape episodes.")

Starting to scrape podcast episodes from: https://open.spotify.com/show/0TBy1wFQocjsf4MGqPVNKV
Page loaded successfully
Clicked 'Load more episodes' button (attempt 1)
Clicked 'Load more episodes' button (attempt 2)
Clicked 'Load more episodes' button (attempt 3)
Clicked 'Load more episodes' button (attempt 4)
Clicked 'Load more episodes' button (attempt 5)
Clicked 'Load more episodes' button (attempt 6)
No 'Load more episodes' button found, all episodes may be loaded
Found 38 episode containers
Extracted episode 1: TMGP Ep 38 with NO-BS strength and fitness coach Cillian O’Connor AKA DYSFUNCTIONALPATTERNS
Extracted episode 2: TMGP Ep 37 with men’s physique winner, bodybuilder, and lifestyle coach - Jonathan Kantor
Extracted episode 3: TMGP Ep 36 with Emanuel Pescari: Austria’s Strongest Man, MULTIPLE WORLD RECORD TITLE HOLDER & INTERNATIONAL STRONGMAN SUPERSTAR Competitor part 2
Extracted episode 4: TMGP Ep 35 with Emanuel Pescari: Austria’s Strongest Man and Elite Strongman Competito

In [9]:
spotify_episodes

['https://open.spotify.com/episode/7iNTGVONthOQBlkIMpJuma',
 'https://open.spotify.com/episode/2zEhXqqhndisCf7dhvT9wf',
 'https://open.spotify.com/episode/5cVBijng6H9VTnVIatyMXs',
 'https://open.spotify.com/episode/3VfywzPhiwlkmnbJOfyfQL',
 'https://open.spotify.com/episode/5ygZIZd7fD2gVNByDzEmVv',
 'https://open.spotify.com/episode/6yu3QR20x44rqqLbUWDTIO']

In [10]:
spotify_episode_urls = []
for url in spotify_episodes:
  if "episode" in url:
    print(url)
    spotify_episode_urls.append(url)

https://open.spotify.com/episode/7iNTGVONthOQBlkIMpJuma
https://open.spotify.com/episode/2zEhXqqhndisCf7dhvT9wf
https://open.spotify.com/episode/5cVBijng6H9VTnVIatyMXs
https://open.spotify.com/episode/3VfywzPhiwlkmnbJOfyfQL
https://open.spotify.com/episode/5ygZIZd7fD2gVNByDzEmVv
https://open.spotify.com/episode/6yu3QR20x44rqqLbUWDTIO


# Concactenate Info from Apple and Spotify

In [ ]:
def create_episode_info(apple_episodes, spotify_episode_urls):
    episodes = []

    # Ensure matching lengths between apple episodes and spotify urls
    if len(apple_episodes) != len(spotify_episode_urls):
        print("Warning: Mismatched number of Apple episodes and Spotify URLs.")
        return []

    for i in range(len(apple_episodes)):
        episode_info = {
            "title": apple_episodes[i]["title"],
            "description": apple_episodes[i]["description"],
            "urls": {
                "spotify": spotify_episode_urls[i],
                "apple": apple_episodes[i]["link"]
            }
        }
        episodes.append(episode_info)

    return episodes

episodes = create_episode_info(apple_episodes, spotify_episode_urls)


# Format into HTML for WEBSITE

In [ ]:
def print_formatted_episodes_html(episodes):
    html_output = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Episodes</title>
        <style>
            body {
                font-family: Arial, sans-serif;
            }
            .episode-card {
                border: 1px solid #ddd;
                padding: 20px;
                margin: 10px 0;
                border-radius: 8px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            }
            .episode-title {
                font-size: 1.5em;
                font-weight: bold;
                margin-bottom: 10px;
            }
            .episode-subtitle {
                font-size: 1.2em;
                font-weight: bold;
                color: #333;
                margin: 5px 0;
            }
            .episode-description {
                font-size: 1em;
                line-height: 1.6;
                margin: 10px 0;
            }
            .platform-link {
                font-weight: bold;
                text-decoration: none;
                margin-right: 10px;
            }
            .apple-link {
                color: #0073e6;
            }
            .spotify-link {
                color: #1DB954;
            }
        </style>
    </head>
    <body>
    <div id="episodes-container">
    """

    # Generate HTML for each episode in reverse order
    for i, episode in enumerate(reversed(episodes), 1):  # Latest to oldest order
        title = episode["title"]
        description = episode["description"]
        apple_url = episode["urls"]["apple"]
        spotify_url = episode["urls"]["spotify"]

        html_output += f"""
        <div class="episode-card">
            <div class="episode-title">Episode {i}</div>
            <div class="episode-subtitle">{title}</div>
            <div class="episode-description">{description}</div>
            <p>
                <a href="{apple_url}" target="_blank" class="platform-link apple-link">
                    🍏 Listen on Apple
                </a>
                <a href="{spotify_url}" target="_blank" class="platform-link spotify-link">
                    🎶 Listen on Spotify
                </a>
            </p>
        </div>
        """

    # Close the main container and body tags
    html_output += """
    </div>
    </body>
    </html>
    """

    print(html_output)

# Example usage:
episodes = create_episode_info(apple_episodes, spotify_episode_urls)
print_formatted_episodes_html(episodes)
